# Calculate Stats on Tuning Distributions

In [5]:
import os
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import scikit_posthocs as posthoc
from statsmodels.stats.multitest import multipletests
from hmmlearn import hmm

sys.path.insert(0, os.path.abspath(r'C:/Users/mmccann/repos/bonhoeffer/prey_capture/'))
import functions_plotting as fp

In [6]:
def h5_to_dict(filepath):
    with pd.HDFStore(filepath, 'r') as store:
        keys = store.keys()
        data = {key[1:]: store[key] for key in keys}
    return data


def kruskal_w_dunn(data, nan_policy='omit', alpha=0.05):
    statistic, p_val = stats.kruskal(*data, nan_policy=nan_policy)
    dunn_test = posthoc.posthoc_dunn([*data], p_adjust='holm')

    # Return just the corrected pvals instead of a whole matrix
    if dunn_test.shape[-1] > 2:
        p_val_corrected = np.array([dunn_test.iloc[0, i] for i in range(1, dunn_test.shape[-1])] + [dunn_test.iloc[1, -1]])
    else:
        p_val_corrected = np.array([dunn_test.iloc[0, i] for i in range(1, dunn_test.shape[-1])])
    return p_val, p_val_corrected


def mannwhitneyu_w_bonferroni(data, nan_policy='omit', alpha=0.05):
    statistic, p_val = stats.mannwhitneyu(*data, nan_policy=nan_policy)
    reject, p_val_corrected, _, _ = multipletests(p_val, alpha=alpha, method='holm')
    return p_val, p_val_corrected


def wilcoxon_w_bonferroni(data, nan_policy='omit', alpha=0.05):
    statistic, p_val = stats.wilcoxon(*data, nan_policy=nan_policy)
    reject, p_val_corrected, _, _ = multipletests(p_val, alpha=alpha, method='holm')
    return p_val, p_val_corrected


def sample_df(data1, data2):
    df1 = data1.dropna()
    df2 = data2.dropna() 

    if len(df1) > len(df2):
        df1 = df1.sample(len(df2), replace=True)
    elif len(df2) > len(df1):
        df2 = df2.sample(len(df1), replace=True)
    else:
        pass

    return df1, df2


def bootstrap_tests(datasets, key, tuning_kind, method, n_iters=10000, alpha=0.05, bootstrap_all=False):
    data = [ds[key][tuning_kind].dropna() for ds in datasets]

    # First run the test on the raw data
    if 'wilcoxon' in method.__name__:
        raw_test = method(sample_df(*data), alpha=alpha)
    elif 'mannwhitneyu' in method.__name__:
        raw_test = method(data, alpha=alpha)
    else:
        raw_test = method(data, alpha=alpha)

    # Now run the bootstrapping
    p_vals = np.zeros(n_iters)
    p_vals_corrected = np.zeros((n_iters, len(data)))

    sample_size = len(max(data, key=len))

    # Run bootstrap
    for i in range(n_iters):

        if bootstrap_all:
            sample_data = [d.sample(sample_size, replace=True) for d in data]
            # sample_data1  = data2.sample(sample_size, replace=True)
        else:
            sample_data = [data[0]] + [d.sample(sample_size, replace=True) for d in data[1:]]
            
        p_val, p_val_corrected = method(sample_data, alpha=alpha)
        p_vals[i] = p_val
        p_vals_corrected[i, :] = p_val_corrected

    # 95% confidence interval of pvals
    ci = stats.t.interval(0.95, len(p_vals) - 1, loc=np.nanmean(p_vals), scale=stats.sem(p_vals, nan_policy='omit'))

    if method == kruskal_w_dunn:
        # Since this is for multiple groups, need to run this column-wise

        # Corrected 95% CIs
        ci_corrected = np.zeros((len(data), 2))
        for i in range(p_vals_corrected.shape[-1]):
            p_val_corr_col = p_vals_corrected[:, i]
            ci_col = stats.t.interval(0.95, len(p_val_corr_col) - 1, loc=np.nanmean(p_val_corr_col), scale=stats.sem(p_val_corr_col, nan_policy='omit'))
            ci_corrected[i, :] = ci_col
    else:
        ci_corrected = stats.t.interval(0.95, len(p_vals_corrected) - 1, loc=np.nanmean(p_vals_corrected), scale=stats.sem(p_vals_corrected, nan_policy='omit'))

    return raw_test, (p_vals, ci), (p_vals_corrected, ci_corrected)


def plot_bootstrap_distributions(data):
    # Plot the bootstrap distributions
    num_ds = len(data)
    cols = 3
    rows = num_ds//cols + 1
    fig, axes = plt.subplots(rows, cols, figsize=(5*cols, 5*rows))

    for i, (ds, ax) in enumerate(zip(data, axes.ravel())):

        raw_test = ds[0]
        bootstrap_tests = ds[1]
        corrected_bootstrap_tests = ds[2]

        ax.hist(bootstrap_tests[0], bins=20)
        ax.axvline(np.percentile(bootstrap_tests[0], 95), 0, 50, color='r')
        ax.axvline(bootstrap_tests[1][0], 0, 50, color='g')
        ax.axvline(bootstrap_tests[1][1], 0, 50, color='g')
        ax.axvline(raw_test[0], 0, 100, color='k')


    return fig, axes

In [7]:
basedir = r"Z:\Prey_capture\WF_Figures"
still_or_all = 'full'
main_exp_path = os.path.join(basedir, still_or_all, r"multi_normal_ALL\stats.hdf5")
control_light_path  = os.path.join(basedir, still_or_all, r"control_normal_ALL\stats.hdf5")
control_dark_path = os.path.join(basedir, still_or_all, r"control_dark_ALL\stats.hdf5")
repeat_fixed_path = os.path.join(basedir, still_or_all, r"repeat_normal_VWheelWF\stats.hdf5")
repeat_free_path = os.path.join(basedir, still_or_all, r"repeat_normal_VTuningWF\stats.hdf5")
output_path = os.path.join(basedir, 'stats', f"{still_or_all}_tuning_stats.h5")

In [8]:
main_exp_data = h5_to_dict(main_exp_path)
control_light_data = h5_to_dict(control_light_path)
control_dark_data = h5_to_dict(control_dark_path)
repeat_fixed_data = h5_to_dict(repeat_fixed_path)
repeat_free_data = h5_to_dict(repeat_free_path)

In [9]:
method = kruskal_w_dunn
bootstrap_iters = 1000

In [10]:
# main_exp_data['frac_vis_resp_fixed'].set_index(['mouse', 'day']).iloc[:10, :]

In [11]:
# main_exp_data['frac_vis_resp_free'].count()

In [12]:
# control_dark_data['frac_kinem_resp_fixed'].count()

In [13]:
# control_light_data['frac_kinem_resp_fixed'].count()

# Kinematic Tuning

In [14]:
def kinem_stats(data, ds_name, alpha=0.05):
    cols = data[0][ds_name].columns.to_list()

    stats = {}
    for col in cols:
        stats[col] = bootstrap_tests(data, ds_name, col,   
                                      kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)

    return stats

In [15]:
# with pd.HDFStore(output_path, 'a') as store:
#     a = store['kruskal_kinem_tuning_free']

#     for var in a.index.to_list():
#         print(var)
#         print(a.loc[var, 'control_dark'][-1])

In [16]:
# Fixed data
fixed_kinem = kinem_stats([main_exp_data, control_light_data, control_dark_data], 'frac_kinem_resp_fixed')
kinem_tuning_fixed = pd.DataFrame(fixed_kinem).T
kinem_tuning_fixed.columns = ['main', 'control_light', 'control_dark']

# Save it
with pd.HDFStore(output_path, 'a') as store:
    store[f'kruskal_kinem_tuning_fixed'] = kinem_tuning_fixed


# Free data
free_kinem = kinem_stats([main_exp_data, control_light_data, control_dark_data], 'frac_kinem_resp_free')
kinem_tuning_free = pd.DataFrame(free_kinem).T
kinem_tuning_free.columns = ['main', 'control_light', 'control_dark']

# Save it
with pd.HDFStore(output_path, 'a') as store:
    store[f'kruskal_kinem_tuning_free'] = kinem_tuning_free


C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\4136049986.py:8: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['main', 'control_light', 'control_dark'], dtype='object')]

  store[f'kruskal_kinem_tuning_fixed'] = kinem_tuning_fixed
C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\4136049986.py:18: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['main', 'control_light', 'control_dark'], dtype='object')]

  store[f'kruskal_kinem_tuning_free'] = kinem_tuning_free


# Visual Tuning

In [17]:
vis_classes = ['', 'mod_', 'not_']

## Frac Visual Tuning - Fixed to Free

### Kruskal-Wallis

In [18]:
for vis_class in vis_classes:
    # Do a Kruskal-Wallis test with posthoc Dunn test for multiple comparisons

    # Visual
    exp_fixed_free_vis = kruskal_w_dunn(sample_df(main_exp_data['frac_vis_resp_fixed'][f'{vis_class}visual'], 
                                                  main_exp_data['frac_vis_resp_free'][f'{vis_class}visual'])
                                                  )
    print('Fixed vs Free Visual:', exp_fixed_free_vis)


    # Orientation
    exp_fixed_free_ori = kruskal_w_dunn(sample_df(main_exp_data['frac_vis_resp_fixed'][f'{vis_class}orientation'], 
                                                  main_exp_data['frac_vis_resp_free'][f'{vis_class}orientation'])
                                                  )
    print('Fixed vs Free Orientation:', exp_fixed_free_ori)


    # Direction
    exp_fixed_free_dir = kruskal_w_dunn(sample_df(main_exp_data['frac_vis_resp_fixed'][f'{vis_class}direction'], 
                                                  main_exp_data['frac_vis_resp_free'][f'{vis_class}direction'])
                                                  )
    print('Fixed vs Free Direction:', exp_fixed_free_dir)

    # Save the data
    with pd.HDFStore(output_path, 'a') as store:
        store[f'{vis_class}vis/kruskal_exp_fixed_free_vis'] = pd.DataFrame(exp_fixed_free_vis, index=['p_val', 'p_val_dunn_posthoc'])
        store[f'{vis_class}vis/kruskal_exp_fixed_free_ori'] = pd.DataFrame(exp_fixed_free_ori, index=['p_val', 'p_val_dunn_posthoc'])
        store[f'{vis_class}vis/kruskal_exp_fixed_free_dir'] = pd.DataFrame(exp_fixed_free_dir, index=['p_val', 'p_val_dunn_posthoc'])

Fixed vs Free Visual: (0.44939864646157335, array([0.44939865]))
Fixed vs Free Orientation: (6.883752847305841e-07, array([6.88375285e-07]))
Fixed vs Free Direction: (0.10380080551334125, array([0.10380081]))


C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\512116551.py:26: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store[f'{vis_class}vis/kruskal_exp_fixed_free_vis'] = pd.DataFrame(exp_fixed_free_vis, index=['p_val', 'p_val_dunn_posthoc'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\512116551.py:27: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store[f'{vis_class}vis/kruskal_exp_fixed_free_ori'] = pd.DataFrame(exp_fixed_free_ori, index=['p_val', 'p_val_dunn_posthoc'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\512116551.py:28: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-ty

Fixed vs Free Visual: (0.00045743224653777936, array([0.00045743]))
Fixed vs Free Orientation: (0.13129093973109643, array([0.13129094]))
Fixed vs Free Direction: (0.08756695399605391, array([0.08756695]))


C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\512116551.py:26: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store[f'{vis_class}vis/kruskal_exp_fixed_free_vis'] = pd.DataFrame(exp_fixed_free_vis, index=['p_val', 'p_val_dunn_posthoc'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\512116551.py:27: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store[f'{vis_class}vis/kruskal_exp_fixed_free_ori'] = pd.DataFrame(exp_fixed_free_ori, index=['p_val', 'p_val_dunn_posthoc'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\512116551.py:28: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-ty

Fixed vs Free Visual: (3.26125836043493e-06, array([3.26125836e-06]))
Fixed vs Free Orientation: (0.0001937325416675049, array([0.00019373]))
Fixed vs Free Direction: (0.0011966217300060356, array([0.00119662]))


C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\512116551.py:26: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store[f'{vis_class}vis/kruskal_exp_fixed_free_vis'] = pd.DataFrame(exp_fixed_free_vis, index=['p_val', 'p_val_dunn_posthoc'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\512116551.py:27: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store[f'{vis_class}vis/kruskal_exp_fixed_free_ori'] = pd.DataFrame(exp_fixed_free_ori, index=['p_val', 'p_val_dunn_posthoc'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\512116551.py:28: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-ty

### Wilcoxon Rank Sum

In [19]:
# Do a Wilcox Rank-Sum test with Bonferroni correction

for vis_class in vis_classes:
    # Visual
    exp_fixed_free_vis = wilcoxon_w_bonferroni(sample_df(main_exp_data['frac_vis_resp_fixed'][f'{vis_class}visual'], 
                                                        main_exp_data['frac_vis_resp_free'][f'{vis_class}visual']))
    print('Fixed vs Free Visual:', exp_fixed_free_vis)


    # Orientation
    exp_fixed_free_ori = wilcoxon_w_bonferroni(sample_df(main_exp_data['frac_vis_resp_fixed'][f'{vis_class}orientation'], 
                                                        main_exp_data['frac_vis_resp_free'][f'{vis_class}orientation']))
    print('Fixed vs Free Orientation:', exp_fixed_free_ori)


    # Direction
    exp_fixed_free_dir = wilcoxon_w_bonferroni(sample_df(main_exp_data['frac_vis_resp_fixed'][f'{vis_class}direction'], 
                                                        main_exp_data['frac_vis_resp_free'][f'{vis_class}direction']))
    print('Fixed vs Free Direction:', exp_fixed_free_dir)

    # Save the data
    with pd.HDFStore(output_path, 'a') as store:
        store[f'{vis_class}vis/wilcoxon_exp_fixed_free_vis'] = pd.DataFrame(exp_fixed_free_vis, index=['p_val', 'p_val_bonferroni'])
        store[f'{vis_class}vis/wilcoxon_exp_fixed_free_ori'] = pd.DataFrame(exp_fixed_free_ori, index=['p_val', 'p_val_bonferroni'])
        store[f'{vis_class}vis/wilcoxon_exp_fixed_free_dir'] = pd.DataFrame(exp_fixed_free_dir, index=['p_val', 'p_val_bonferroni'])

Fixed vs Free Visual: (0.8957222313738202, array([0.89572223]))
Fixed vs Free Orientation: (5.6243341243348525e-08, array([5.62433412e-08]))
Fixed vs Free Direction: (0.04933242463327135, array([0.04933242]))


C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\4259356661.py:23: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store[f'{vis_class}vis/wilcoxon_exp_fixed_free_vis'] = pd.DataFrame(exp_fixed_free_vis, index=['p_val', 'p_val_bonferroni'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\4259356661.py:24: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store[f'{vis_class}vis/wilcoxon_exp_fixed_free_ori'] = pd.DataFrame(exp_fixed_free_ori, index=['p_val', 'p_val_bonferroni'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\4259356661.py:25: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-t

Fixed vs Free Visual: (0.0001936225336793642, array([0.00019362]))
Fixed vs Free Orientation: (0.09841049145651927, array([0.09841049]))
Fixed vs Free Direction: (0.148096669435202, array([0.14809667]))


C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\4259356661.py:23: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store[f'{vis_class}vis/wilcoxon_exp_fixed_free_vis'] = pd.DataFrame(exp_fixed_free_vis, index=['p_val', 'p_val_bonferroni'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\4259356661.py:24: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store[f'{vis_class}vis/wilcoxon_exp_fixed_free_ori'] = pd.DataFrame(exp_fixed_free_ori, index=['p_val', 'p_val_bonferroni'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\4259356661.py:25: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-t

Fixed vs Free Visual: (2.7780197254206182e-09, array([2.77801973e-09]))
Fixed vs Free Orientation: (4.015643462997091e-06, array([4.01564346e-06]))
Fixed vs Free Direction: (2.9346859614647896e-05, array([2.93468596e-05]))


C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\4259356661.py:23: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store[f'{vis_class}vis/wilcoxon_exp_fixed_free_vis'] = pd.DataFrame(exp_fixed_free_vis, index=['p_val', 'p_val_bonferroni'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\4259356661.py:24: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store[f'{vis_class}vis/wilcoxon_exp_fixed_free_ori'] = pd.DataFrame(exp_fixed_free_ori, index=['p_val', 'p_val_bonferroni'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\4259356661.py:25: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-t

### Mann-Whitney U

In [20]:
# Do a Mann-Whitney U test with Bonferroni correction
for vis_class in vis_classes:
    # Visual
    exp_fixed_free_vis = mannwhitneyu_w_bonferroni(sample_df(main_exp_data['frac_vis_resp_fixed'][f'{vis_class}visual'], 
                                                main_exp_data['frac_vis_resp_free'][f'{vis_class}visual']))
    print('Fixed vs Free Visual:', exp_fixed_free_vis)

    # Orientation
    exp_fixed_free_ori = mannwhitneyu_w_bonferroni(sample_df(main_exp_data['frac_vis_resp_fixed'][f'{vis_class}orientation'], 
                                                main_exp_data['frac_vis_resp_free'][f'{vis_class}orientation']))
    print('Fixed vs Free Orientation:', exp_fixed_free_ori)

    # Direction
    exp_fixed_free_dir = mannwhitneyu_w_bonferroni(sample_df(main_exp_data['frac_vis_resp_fixed'][f'{vis_class}direction'], 
                                                main_exp_data['frac_vis_resp_free'][f'{vis_class}direction']))
    print('Fixed vs Free Direction:', exp_fixed_free_dir)

    # Save the data
    with pd.HDFStore(output_path, 'a') as store:
        store[f'{vis_class}vis/mannwhitneyu_exp_fixed_free_vis'] = pd.DataFrame(exp_fixed_free_vis, index=['p_val', 'p_val_bonferroni'])
        store[f'{vis_class}vis/mannwhitneyu_exp_fixed_free_ori'] = pd.DataFrame(exp_fixed_free_ori, index=['p_val', 'p_val_bonferroni'])
        store[f'{vis_class}vis/mannwhitneyu_exp_fixed_free_dir'] = pd.DataFrame(exp_fixed_free_dir, index=['p_val', 'p_val_bonferroni'])

Fixed vs Free Visual: (0.4502294377882633, array([0.45022944]))
Fixed vs Free Orientation: (6.933893977149523e-07, array([6.93389398e-07]))
Fixed vs Free Direction: (0.10409758705244772, array([0.10409759]))


C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\1293581978.py:20: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store[f'{vis_class}vis/mannwhitneyu_exp_fixed_free_vis'] = pd.DataFrame(exp_fixed_free_vis, index=['p_val', 'p_val_bonferroni'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\1293581978.py:21: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store[f'{vis_class}vis/mannwhitneyu_exp_fixed_free_ori'] = pd.DataFrame(exp_fixed_free_ori, index=['p_val', 'p_val_bonferroni'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\1293581978.py:22: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directl

Fixed vs Free Visual: (0.000459816775601058, array([0.00045982]))
Fixed vs Free Orientation: (0.13164672020480175, array([0.13164672]))
Fixed vs Free Direction: (0.08782418337700752, array([0.08782418]))


C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\1293581978.py:20: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store[f'{vis_class}vis/mannwhitneyu_exp_fixed_free_vis'] = pd.DataFrame(exp_fixed_free_vis, index=['p_val', 'p_val_bonferroni'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\1293581978.py:21: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store[f'{vis_class}vis/mannwhitneyu_exp_fixed_free_ori'] = pd.DataFrame(exp_fixed_free_ori, index=['p_val', 'p_val_bonferroni'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\1293581978.py:22: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directl

Fixed vs Free Visual: (3.283338042575187e-06, array([3.28333804e-06]))
Fixed vs Free Orientation: (0.0001949594692131118, array([0.00019496]))
Fixed vs Free Direction: (0.00120268863930236, array([0.00120269]))


C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\1293581978.py:20: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store[f'{vis_class}vis/mannwhitneyu_exp_fixed_free_vis'] = pd.DataFrame(exp_fixed_free_vis, index=['p_val', 'p_val_bonferroni'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\1293581978.py:21: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store[f'{vis_class}vis/mannwhitneyu_exp_fixed_free_ori'] = pd.DataFrame(exp_fixed_free_ori, index=['p_val', 'p_val_bonferroni'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\1293581978.py:22: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directl

## Frac Visual Tuning - Exp vs Controls

### Fixed Kruskal Wallis

In [21]:
# Do a Kruskal-Wallis test with Dunn posthoc
for vis_class in vis_classes:
    # Visual
    frac_fixed_vis = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                    'frac_vis_resp_fixed', 'visual',
                                    kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
    print('Fixed vs Controls Visual 95% CI', frac_fixed_vis[1][1])
    print('Fixed vs Controls Visual 95% CI - Corrected:', frac_fixed_vis[-1][1])
    print('Fixed vs Controls Visual Raw:', frac_fixed_vis[0])

    # Orientation
    frac_fixed_ori = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                    'frac_vis_resp_fixed', 'orientation',
                                    kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
    print('Fixed vs Controls Ori 95% CI', frac_fixed_ori[1][1])
    print('Fixed vs Controls Ori 95% CI - Corrected:', frac_fixed_ori[-1][1])
    print('Fixed vs Controls Ori Raw:', frac_fixed_ori[0])

    # Direction
    frac_fixed_dir = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                    'frac_vis_resp_fixed', 'direction',
                                    kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
    print('Fixed vs Controls Dir 95% CI', frac_fixed_dir[1][1])
    print('Fixed vs Controls Dir 95% CI - Corrected:', frac_fixed_dir[-1][1])
    print('Fixed vs Controls Dir Raw:', frac_fixed_dir[0])

    # Save the data
    with pd.HDFStore(output_path, 'a') as store:
        store[f'{vis_class}vis/kruskal_frac_fixed_vis'] = pd.DataFrame(list(frac_fixed_vis), columns=['pval', 'ci'],
                                                    index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T
        store[f'{vis_class}vis/kruskal_frac_fixed_ori'] = pd.DataFrame(list(frac_fixed_ori), columns=['pval', 'ci'],
                                                    index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T
        store[f'{vis_class}vis/kruskal_frac_fixed_dir'] = pd.DataFrame(list(frac_fixed_dir), columns=['pval', 'ci'],
                                                    index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T

Fixed vs Controls Visual 95% CI (2.5417322008283457e-11, 3.854425051702783e-10)
Fixed vs Controls Visual 95% CI - Corrected: [[-1.63262691e-07  1.13748285e-06]
 [ 1.97833718e-10  2.82499261e-09]
 [ 3.81610454e-01  4.17783414e-01]]
Fixed vs Controls Visual Raw: (3.2014409264734207e-06, array([2.67957738e-03, 6.36704690e-05, 6.06531979e-01]))
Fixed vs Controls Ori 95% CI (1.6644244762646192e-24, 1.6282858196387512e-22)
Fixed vs Controls Ori 95% CI - Corrected: [[ 3.65387799e-20  3.87206626e-19]
 [-1.27978710e-16  6.22121576e-16]
 [ 5.04846152e-01  5.35052313e-01]]
Fixed vs Controls Ori Raw: (3.299103554764786e-08, array([4.00639943e-05, 1.66410525e-05, 9.09774218e-01]))
Fixed vs Controls Dir 95% CI (-1.220901706729164e-10, 3.8575526083479863e-10)
Fixed vs Controls Dir 95% CI - Corrected: [[-3.65550299e-10  1.23375473e-09]
 [ 2.85732598e-08  5.00263108e-07]
 [ 1.80432969e-01  2.05848704e-01]]
Fixed vs Controls Dir Raw: (7.753844837840536e-06, array([6.03607635e-04, 7.29513186e-04, 7.01426

C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\1589276187.py:29: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests'], dtype='object')]

  store[f'{vis_class}vis/kruskal_frac_fixed_vis'] = pd.DataFrame(list(frac_fixed_vis), columns=['pval', 'ci'],
C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\1589276187.py:31: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests'], dtype='object')]

  store[f'{vis_class}vis/kruskal_frac_fixed_ori'] = pd.DataFrame(list(frac_fixed_ori), columns=['pval', 'ci'],
C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\1589276187.py:33: PerformanceWarning: 
your performance may suff

Fixed vs Controls Visual 95% CI (-2.341298557813793e-11, 1.405904842203229e-10)
Fixed vs Controls Visual 95% CI - Corrected: [[ 7.55951277e-08  4.20184681e-07]
 [-2.82195146e-11  1.52841787e-09]
 [ 3.81567775e-01  4.18474745e-01]]
Fixed vs Controls Visual Raw: (3.2014409264734207e-06, array([2.67957738e-03, 6.36704690e-05, 6.06531979e-01]))
Fixed vs Controls Ori 95% CI (-2.6127034165767524e-24, 1.5871818962960903e-23)
Fixed vs Controls Ori 95% CI - Corrected: [[-3.14047420e-19  1.04312326e-18]
 [-3.87881065e-17  2.24143028e-16]
 [ 5.17424792e-01  5.48171950e-01]]
Fixed vs Controls Ori Raw: (3.299103554764786e-08, array([4.00639943e-05, 1.66410525e-05, 9.09774218e-01]))
Fixed vs Controls Dir 95% CI (-1.0121823439960787e-12, 1.745835390082729e-11)
Fixed vs Controls Dir 95% CI - Corrected: [[-1.09350512e-11  1.03331703e-10]
 [-1.07861009e-07  1.47031399e-06]
 [ 1.85538286e-01  2.11814127e-01]]
Fixed vs Controls Dir Raw: (7.753844837840536e-06, array([6.03607635e-04, 7.29513186e-04, 7.0142

C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\1589276187.py:29: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests'], dtype='object')]

  store[f'{vis_class}vis/kruskal_frac_fixed_vis'] = pd.DataFrame(list(frac_fixed_vis), columns=['pval', 'ci'],
C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\1589276187.py:31: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests'], dtype='object')]

  store[f'{vis_class}vis/kruskal_frac_fixed_ori'] = pd.DataFrame(list(frac_fixed_ori), columns=['pval', 'ci'],
C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\1589276187.py:33: PerformanceWarning: 
your performance may suff

Fixed vs Controls Visual 95% CI (-6.255093521326121e-11, 7.133039165938428e-10)
Fixed vs Controls Visual 95% CI - Corrected: [[-1.52555330e-07  5.18379479e-06]
 [ 1.30818422e-10  9.76072815e-10]
 [ 3.90735746e-01  4.27931310e-01]]
Fixed vs Controls Visual Raw: (3.2014409264734207e-06, array([2.67957738e-03, 6.36704690e-05, 6.06531979e-01]))
Fixed vs Controls Ori 95% CI (-6.172679309828902e-23, 2.4112579522974436e-22)
Fixed vs Controls Ori 95% CI - Corrected: [[-3.29899626e-19  1.34898504e-18]
 [-1.43124914e-17  1.02353268e-16]
 [ 5.13204288e-01  5.43537821e-01]]
Fixed vs Controls Ori Raw: (3.299103554764786e-08, array([4.00639943e-05, 1.66410525e-05, 9.09774218e-01]))
Fixed vs Controls Dir 95% CI (-1.0339043269176514e-12, 2.2913052735723917e-11)
Fixed vs Controls Dir 95% CI - Corrected: [[ 8.92686114e-12  6.52905027e-11]
 [-1.59628942e-08  3.00539148e-07]
 [ 1.87628092e-01  2.14247251e-01]]
Fixed vs Controls Dir Raw: (7.753844837840536e-06, array([6.03607635e-04, 7.29513186e-04, 7.0142

C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\1589276187.py:29: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests'], dtype='object')]

  store[f'{vis_class}vis/kruskal_frac_fixed_vis'] = pd.DataFrame(list(frac_fixed_vis), columns=['pval', 'ci'],
C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\1589276187.py:31: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests'], dtype='object')]

  store[f'{vis_class}vis/kruskal_frac_fixed_ori'] = pd.DataFrame(list(frac_fixed_ori), columns=['pval', 'ci'],
C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\1589276187.py:33: PerformanceWarning: 
your performance may suff

In [22]:
# with pd.HDFStore(output_path, 'a') as store:
#     a = store[f'{vis_class}vis/kruskal_frac_fixed_dir']
#     print(a.loc['ci', 'corrected_bootstrap_tests'])

In [23]:
# frac_fixed_fig = plot_bootstrap_distributions([frac_fixed_gen, 
#                                                frac_fixed_vis, frac_fixed_vis_no_gen, 
#                                                frac_fixed_ori, frac_fixed_ori_no_gen,
#                                                frac_fixed_dir, frac_fixed_dir_no_gen])
# plt.show()

### Free Kruskal Wallis

In [24]:
# Do a Kruskal-Wallis test with Dunn posthoc
for vis_class in vis_classes:
    # Visual
    frac_free_vis = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                    'frac_vis_resp_free', 'visual',
                                    kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
    print('Free vs Controls Visual 95% CI', frac_free_vis[1][1])
    print('Free vs Controls Visual 95% CI - Corrected:', frac_free_vis[-1][1])
    print('Free vs Controls Visual Raw:', frac_free_vis[0])

    # Orientation
    frac_free_ori = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                    'frac_vis_resp_free', 'orientation',
                                    kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
    print('Free vs Controls Ori 95% CI', frac_free_ori[1][1])
    print('Free vs Controls Ori 95% CI - Corrected:', frac_free_ori[-1][1])
    print('Free vs Controls Ori Raw:', frac_free_ori[0])

    # Direction
    frac_free_dir = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                    'frac_vis_resp_free', 'direction',
                                    kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
    print('Free vs Controls Dir 95% CI', frac_free_dir[1][1])
    print('Free vs Controls Dir 95% CI - Corrected:', frac_free_dir[-1][1])
    print('Free vs Controls Dir Raw:', frac_free_dir[0])


    # Save the data
    with pd.HDFStore(output_path, 'a') as store:
        store[f'{vis_class}vis/kruskal_frac_free_vis'] = pd.DataFrame(list(frac_free_vis), columns=['pval', 'ci'],
                                                    index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T
        store[f'{vis_class}vis/kruskal_frac_free_ori'] = pd.DataFrame(list(frac_free_ori), columns=['pval', 'ci'],
                                                    index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T
        store[f'{vis_class}vis/kruskal_frac_free_dir'] = pd.DataFrame(list(frac_free_dir), columns=['pval', 'ci'],
                                                    index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T

Free vs Controls Visual 95% CI (-5.3144337899489904e-08, 3.5789710004661864e-07)
Free vs Controls Visual 95% CI - Corrected: [[ 5.11513495e-08  3.16696325e-06]
 [-5.86771287e-07  3.27670726e-06]
 [ 5.02046501e-01  5.35928823e-01]]
Free vs Controls Visual Raw: (1.6781978854705825e-05, array([2.14577042e-03, 5.32204021e-04, 9.17489829e-01]))
Free vs Controls Ori 95% CI (3.065740034231262e-08, 4.37116018742714e-07)
Free vs Controls Ori 95% CI - Corrected: [[5.58119374e-04 2.91441293e-03]
 [4.78035852e-09 3.61178872e-07]
 [4.27377793e-02 5.45136191e-02]]
Free vs Controls Ori Raw: (0.0006527808927983592, array([0.05294006, 0.00218761, 0.50272113]))
Free vs Controls Dir 95% CI (-3.5460270411695664e-07, 1.9200281281057023e-06)
Free vs Controls Dir 95% CI - Corrected: [[ 5.32962078e-04  2.50767880e-03]
 [-3.18549667e-07  1.50989732e-06]
 [ 2.32682103e-02  3.13856522e-02]]
Free vs Controls Dir Raw: (0.00025095700950984116, array([0.04135328, 0.00089175, 0.44289122]))


C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\361439579.py:30: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests'], dtype='object')]

  store[f'{vis_class}vis/kruskal_frac_free_vis'] = pd.DataFrame(list(frac_free_vis), columns=['pval', 'ci'],
C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\361439579.py:32: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests'], dtype='object')]

  store[f'{vis_class}vis/kruskal_frac_free_ori'] = pd.DataFrame(list(frac_free_ori), columns=['pval', 'ci'],
C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\361439579.py:34: PerformanceWarning: 
your performance may suffer as P

Free vs Controls Visual 95% CI (-6.309107136802632e-08, 2.161105136141584e-07)
Free vs Controls Visual 95% CI - Corrected: [[-7.76156035e-09  3.41600181e-06]
 [-1.27683341e-07  6.95770541e-07]
 [ 5.09709689e-01  5.43708346e-01]]
Free vs Controls Visual Raw: (1.6781978854705825e-05, array([2.14577042e-03, 5.32204021e-04, 9.17489829e-01]))
Free vs Controls Ori 95% CI (1.9767317120795692e-08, 1.2160960335478674e-07)
Free vs Controls Ori 95% CI - Corrected: [[5.31921701e-04 1.16484304e-03]
 [1.29262418e-08 7.59801120e-08]
 [4.17337820e-02 5.30532120e-02]]
Free vs Controls Ori Raw: (0.0006527808927983592, array([0.05294006, 0.00218761, 0.50272113]))
Free vs Controls Dir 95% CI (2.8595078034094126e-09, 1.245988916250625e-07)
Free vs Controls Dir 95% CI - Corrected: [[ 5.91407564e-04  1.15130986e-03]
 [-2.58884513e-08  1.70540437e-07]
 [ 2.50653461e-02  3.37932373e-02]]
Free vs Controls Dir Raw: (0.00025095700950984116, array([0.04135328, 0.00089175, 0.44289122]))


C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\361439579.py:30: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests'], dtype='object')]

  store[f'{vis_class}vis/kruskal_frac_free_vis'] = pd.DataFrame(list(frac_free_vis), columns=['pval', 'ci'],
C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\361439579.py:32: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests'], dtype='object')]

  store[f'{vis_class}vis/kruskal_frac_free_ori'] = pd.DataFrame(list(frac_free_ori), columns=['pval', 'ci'],
C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\361439579.py:34: PerformanceWarning: 
your performance may suffer as P

Free vs Controls Visual 95% CI (1.3734940883784355e-09, 4.7152617703566516e-08)
Free vs Controls Visual 95% CI - Corrected: [[3.04706616e-07 1.07617144e-06]
 [2.33785892e-08 6.91831875e-07]
 [4.98748201e-01 5.33215354e-01]]
Free vs Controls Visual Raw: (1.6781978854705825e-05, array([2.14577042e-03, 5.32204021e-04, 9.17489829e-01]))
Free vs Controls Ori 95% CI (-9.44359921087845e-09, 1.9317716874366347e-07)
Free vs Controls Ori 95% CI - Corrected: [[ 6.55248134e-04  1.43430175e-03]
 [-1.34798129e-09  1.08231471e-07]
 [ 3.83085079e-02  4.77958694e-02]]
Free vs Controls Ori Raw: (0.0006527808927983592, array([0.05294006, 0.00218761, 0.50272113]))
Free vs Controls Dir 95% CI (-8.600124954092861e-08, 3.568229681942717e-07)
Free vs Controls Dir 95% CI - Corrected: [[ 6.71526620e-04  1.25595040e-03]
 [-9.52567234e-08  3.61416573e-07]
 [ 2.47709918e-02  3.42302594e-02]]
Free vs Controls Dir Raw: (0.00025095700950984116, array([0.04135328, 0.00089175, 0.44289122]))


C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\361439579.py:30: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests'], dtype='object')]

  store[f'{vis_class}vis/kruskal_frac_free_vis'] = pd.DataFrame(list(frac_free_vis), columns=['pval', 'ci'],
C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\361439579.py:32: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests'], dtype='object')]

  store[f'{vis_class}vis/kruskal_frac_free_ori'] = pd.DataFrame(list(frac_free_ori), columns=['pval', 'ci'],
C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\361439579.py:34: PerformanceWarning: 
your performance may suffer as P

In [25]:
# frac_free_fig = plot_bootstrap_distributions([frac_free_gen, 
#                                               frac_free_vis, frac_free_vis_no_gen, 
#                                               frac_free_ori, frac_free_ori_no_gen,
#                                               frac_free_dir, frac_free_dir_no_gen])
# plt.show()

## Count Vis Tuning - Fixed vs Free

### Kruskal-Wallis

In [26]:
# Do a Kruskal-Wallis test with posthoc Dunn test for multiple comparisons
for vis_class in vis_classes:

    # Visual
    count_exp_fixed_free_vis = kruskal_w_dunn(sample_df(main_exp_data['count_vis_resp_fixed'][f'{vis_class}visual'], 
                                                main_exp_data['count_vis_resp_free'][f'{vis_class}visual']))
    print('Fixed vs Free Visual:', exp_fixed_free_vis)

    # Orientation
    count_exp_fixed_free_ori = kruskal_w_dunn(sample_df(main_exp_data['count_vis_resp_fixed'][f'{vis_class}orientation'], 
                                                main_exp_data['count_vis_resp_free'][f'{vis_class}orientation']))
    print('Fixed vs Free Orientation:', exp_fixed_free_ori)


    # Direction
    count_exp_fixed_free_dir = kruskal_w_dunn(sample_df(main_exp_data['count_vis_resp_fixed'][f'{vis_class}direction'], 
                                                main_exp_data['count_vis_resp_free'][f'{vis_class}direction']))
    print('Fixed vs Free Direction:', exp_fixed_free_dir)

    # Save the data
    with pd.HDFStore(output_path, 'a') as store:
        store[f'{vis_class}vis/kruskal_exp_fixed_free_vis_count'] = pd.DataFrame(count_exp_fixed_free_vis, index=['p_val', 'p_val_dunn_posthoc'])
        store[f'{vis_class}vis/kruskal_exp_fixed_free_ori_count'] = pd.DataFrame(count_exp_fixed_free_ori, index=['p_val', 'p_val_dunn_posthoc'])
        store[f'{vis_class}vis/kruskal_exp_fixed_free_dir_count'] = pd.DataFrame(count_exp_fixed_free_dir, index=['p_val', 'p_val_dunn_posthoc'])

Fixed vs Free Visual: (3.283338042575187e-06, array([3.28333804e-06]))
Fixed vs Free Orientation: (0.0001949594692131118, array([0.00019496]))
Fixed vs Free Direction: (0.00120268863930236, array([0.00120269]))


C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\1890522773.py:22: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store[f'{vis_class}vis/kruskal_exp_fixed_free_vis_count'] = pd.DataFrame(count_exp_fixed_free_vis, index=['p_val', 'p_val_dunn_posthoc'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\1890522773.py:23: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store[f'{vis_class}vis/kruskal_exp_fixed_free_ori_count'] = pd.DataFrame(count_exp_fixed_free_ori, index=['p_val', 'p_val_dunn_posthoc'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\1890522773.py:24: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it 

Fixed vs Free Visual: (3.283338042575187e-06, array([3.28333804e-06]))
Fixed vs Free Orientation: (0.0001949594692131118, array([0.00019496]))
Fixed vs Free Direction: (0.00120268863930236, array([0.00120269]))


C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\1890522773.py:22: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store[f'{vis_class}vis/kruskal_exp_fixed_free_vis_count'] = pd.DataFrame(count_exp_fixed_free_vis, index=['p_val', 'p_val_dunn_posthoc'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\1890522773.py:23: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store[f'{vis_class}vis/kruskal_exp_fixed_free_ori_count'] = pd.DataFrame(count_exp_fixed_free_ori, index=['p_val', 'p_val_dunn_posthoc'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\1890522773.py:24: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it 

Fixed vs Free Visual: (3.283338042575187e-06, array([3.28333804e-06]))
Fixed vs Free Orientation: (0.0001949594692131118, array([0.00019496]))
Fixed vs Free Direction: (0.00120268863930236, array([0.00120269]))


C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\1890522773.py:22: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store[f'{vis_class}vis/kruskal_exp_fixed_free_vis_count'] = pd.DataFrame(count_exp_fixed_free_vis, index=['p_val', 'p_val_dunn_posthoc'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\1890522773.py:23: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store[f'{vis_class}vis/kruskal_exp_fixed_free_ori_count'] = pd.DataFrame(count_exp_fixed_free_ori, index=['p_val', 'p_val_dunn_posthoc'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\1890522773.py:24: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it 

### Wilcoxon Rank Sum

In [27]:
# Do a Wilcox Rank-Sum test with Bonferroni correction
for vis_class in vis_classes:

    # Visual
    count_exp_fixed_free_vis = wilcoxon_w_bonferroni(sample_df(main_exp_data['count_vis_resp_fixed'][f'{vis_class}visual'], 
                                                main_exp_data['count_vis_resp_free'][f'{vis_class}visual']))
    print('Fixed vs Free Visual:', exp_fixed_free_vis)

    # Orientation
    count_exp_fixed_free_ori = wilcoxon_w_bonferroni(sample_df(main_exp_data['count_vis_resp_fixed'][f'{vis_class}orientation'], 
                                                main_exp_data['count_vis_resp_free'][f'{vis_class}orientation']))
    print('Fixed vs Free Orientation:', exp_fixed_free_ori)

    # Direction
    count_exp_fixed_free_dir = wilcoxon_w_bonferroni(sample_df(main_exp_data['count_vis_resp_fixed'][f'{vis_class}direction'], 
                                                main_exp_data['count_vis_resp_free'][f'{vis_class}direction']))
    print('Fixed vs Free Direction:', exp_fixed_free_dir)

    # Save the data
    with pd.HDFStore(output_path, 'a') as store:
        store[f'{vis_class}vis/wilcoxon_exp_fixed_free_vis_count'] = pd.DataFrame(count_exp_fixed_free_vis, index=['p_val', 'p_val_dunn_posthoc'])
        store[f'{vis_class}vis/wilcoxon_exp_fixed_freew_ori_count'] = pd.DataFrame(count_exp_fixed_free_ori, index=['p_val', 'p_val_dunn_posthoc'])
        store[f'{vis_class}vis/wilcoxon_exp_fixed_free_dir_count'] = pd.DataFrame(count_exp_fixed_free_dir, index=['p_val', 'p_val_dunn_posthoc'])

Fixed vs Free Visual: (3.283338042575187e-06, array([3.28333804e-06]))
Fixed vs Free Orientation: (0.0001949594692131118, array([0.00019496]))
Fixed vs Free Direction: (0.00120268863930236, array([0.00120269]))


C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\553109606.py:21: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store[f'{vis_class}vis/wilcoxon_exp_fixed_free_vis_count'] = pd.DataFrame(count_exp_fixed_free_vis, index=['p_val', 'p_val_dunn_posthoc'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\553109606.py:22: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store[f'{vis_class}vis/wilcoxon_exp_fixed_freew_ori_count'] = pd.DataFrame(count_exp_fixed_free_ori, index=['p_val', 'p_val_dunn_posthoc'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\553109606.py:23: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it 

Fixed vs Free Visual: (3.283338042575187e-06, array([3.28333804e-06]))
Fixed vs Free Orientation: (0.0001949594692131118, array([0.00019496]))
Fixed vs Free Direction: (0.00120268863930236, array([0.00120269]))


C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\553109606.py:21: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store[f'{vis_class}vis/wilcoxon_exp_fixed_free_vis_count'] = pd.DataFrame(count_exp_fixed_free_vis, index=['p_val', 'p_val_dunn_posthoc'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\553109606.py:22: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store[f'{vis_class}vis/wilcoxon_exp_fixed_freew_ori_count'] = pd.DataFrame(count_exp_fixed_free_ori, index=['p_val', 'p_val_dunn_posthoc'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\553109606.py:23: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it 

Fixed vs Free Visual: (3.283338042575187e-06, array([3.28333804e-06]))
Fixed vs Free Orientation: (0.0001949594692131118, array([0.00019496]))
Fixed vs Free Direction: (0.00120268863930236, array([0.00120269]))


C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\553109606.py:21: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store[f'{vis_class}vis/wilcoxon_exp_fixed_free_vis_count'] = pd.DataFrame(count_exp_fixed_free_vis, index=['p_val', 'p_val_dunn_posthoc'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\553109606.py:22: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store[f'{vis_class}vis/wilcoxon_exp_fixed_freew_ori_count'] = pd.DataFrame(count_exp_fixed_free_ori, index=['p_val', 'p_val_dunn_posthoc'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\553109606.py:23: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it 

### Mann-Whitney U

In [28]:
# Do a Mann-Whitney U test with Bonferroni correction
for vis_class in vis_classes:

    # Visual
    count_exp_fixed_free_vis = mannwhitneyu_w_bonferroni(sample_df(main_exp_data['count_vis_resp_fixed'][f'{vis_class}visual'], 
                                                main_exp_data['count_vis_resp_free'][f'{vis_class}visual']))
    print('Fixed vs Free Visual:', exp_fixed_free_vis)

    # Orientation
    count_exp_fixed_free_ori = mannwhitneyu_w_bonferroni(sample_df(main_exp_data['count_vis_resp_fixed'][f'{vis_class}orientation'], 
                                                main_exp_data['count_vis_resp_free'][f'{vis_class}orientation']))
    print('Fixed vs Free Orientation:', exp_fixed_free_ori)

    # Direction
    count_exp_fixed_free_dir = mannwhitneyu_w_bonferroni(sample_df(main_exp_data['count_vis_resp_fixed'][f'{vis_class}direction'], 
                                                main_exp_data['count_vis_resp_free'][f'{vis_class}direction']))
    print('Fixed vs Free Direction:', exp_fixed_free_dir)

    # Save the data
    with pd.HDFStore(output_path, 'a') as store:
        store[f'{vis_class}vis/mannwhitneyu_exp_fixed_free_vis_count'] = pd.DataFrame(count_exp_fixed_free_vis, index=['p_val', 'p_val_dunn_posthoc'])
        store[f'{vis_class}vis/mannwhitneyu_exp_fixed_freew_ori_count'] = pd.DataFrame(count_exp_fixed_free_ori, index=['p_val', 'p_val_dunn_posthoc'])
        store[f'{vis_class}vis/mannwhitneyu_exp_fixed_free_dir_count'] = pd.DataFrame(count_exp_fixed_free_dir, index=['p_val', 'p_val_dunn_posthoc'])

Fixed vs Free Visual: (3.283338042575187e-06, array([3.28333804e-06]))
Fixed vs Free Orientation: (0.0001949594692131118, array([0.00019496]))
Fixed vs Free Direction: (0.00120268863930236, array([0.00120269]))


C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\920968691.py:21: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store[f'{vis_class}vis/mannwhitneyu_exp_fixed_free_vis_count'] = pd.DataFrame(count_exp_fixed_free_vis, index=['p_val', 'p_val_dunn_posthoc'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\920968691.py:22: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store[f'{vis_class}vis/mannwhitneyu_exp_fixed_freew_ori_count'] = pd.DataFrame(count_exp_fixed_free_ori, index=['p_val', 'p_val_dunn_posthoc'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\920968691.py:23: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types 

Fixed vs Free Visual: (3.283338042575187e-06, array([3.28333804e-06]))
Fixed vs Free Orientation: (0.0001949594692131118, array([0.00019496]))
Fixed vs Free Direction: (0.00120268863930236, array([0.00120269]))


C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\920968691.py:21: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store[f'{vis_class}vis/mannwhitneyu_exp_fixed_free_vis_count'] = pd.DataFrame(count_exp_fixed_free_vis, index=['p_val', 'p_val_dunn_posthoc'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\920968691.py:22: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store[f'{vis_class}vis/mannwhitneyu_exp_fixed_freew_ori_count'] = pd.DataFrame(count_exp_fixed_free_ori, index=['p_val', 'p_val_dunn_posthoc'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\920968691.py:23: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types 

Fixed vs Free Visual: (3.283338042575187e-06, array([3.28333804e-06]))
Fixed vs Free Orientation: (0.0001949594692131118, array([0.00019496]))
Fixed vs Free Direction: (0.00120268863930236, array([0.00120269]))


C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\920968691.py:21: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store[f'{vis_class}vis/mannwhitneyu_exp_fixed_free_vis_count'] = pd.DataFrame(count_exp_fixed_free_vis, index=['p_val', 'p_val_dunn_posthoc'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\920968691.py:22: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store[f'{vis_class}vis/mannwhitneyu_exp_fixed_freew_ori_count'] = pd.DataFrame(count_exp_fixed_free_ori, index=['p_val', 'p_val_dunn_posthoc'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\920968691.py:23: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types 

## Count Vis Tuning - Exp vs Controls

### Fixed Kruskal Wallis

In [29]:
# Do a Kruskal-Wallis test with Dunn posthoc

for vis_class in vis_classes:

    # Visual
    count_fixed_vis = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                    'count_vis_resp_fixed', 'visual',   
                                    kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
    print('Fixed vs Controls Visual 95% CI:', count_fixed_vis[1][1])
    print('Fixed vs Controls Visual Raw:', count_fixed_vis[0])

    # Orientation
    count_fixed_ori = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                    'count_vis_resp_fixed', 'orientation',
                                    kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
    print('Fixed vs Controls Orientation 95% CI:', count_fixed_ori[1][1])
    print('Fixed vs Controls Orientation Raw:', count_fixed_ori[0])

    # Direction
    count_fixed_dir = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                    'count_vis_resp_fixed', 'direction',
                                    kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
    print('Fixed vs Controls Direction 95% CI:', count_fixed_dir[1][1])
    print('Fixed vs Controls Direction Raw:', count_fixed_dir[0])

    # Save the data
    with pd.HDFStore(output_path, 'a') as store:
        store[f'{vis_class}vis/kruskal_count_fixed_vis'] = pd.DataFrame(list(count_fixed_vis), columns=['pval', 'ci'],
                                                    index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T
        store[f'{vis_class}vis/kruskal_count_fixed_ori'] = pd.DataFrame(list(count_fixed_ori), columns=['pval', 'ci'],
                                                    index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T
        store[f'{vis_class}vis/kruskal_count_fixed_dir'] = pd.DataFrame(list(count_fixed_dir), columns=['pval', 'ci'],
                                                    index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T

Fixed vs Controls Visual 95% CI: (8.987338912598435e-08, 5.751958705100255e-07)
Fixed vs Controls Visual Raw: (0.000268196058097534, array([0.00634131, 0.00634131, 0.78859931]))
Fixed vs Controls Orientation 95% CI: (-5.318967052521085e-23, 2.3578324501800215e-22)
Fixed vs Controls Orientation Raw: (5.443136292223412e-08, array([5.21956459e-05, 2.30637753e-05, 9.07642115e-01]))
Fixed vs Controls Direction 95% CI: (-1.4474948608057018e-10, 4.922230642580338e-10)
Fixed vs Controls Direction Raw: (1.6802916286478037e-05, array([0.00106744, 0.00106744, 0.73690399]))


C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\1713287464.py:28: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests'], dtype='object')]

  store[f'{vis_class}vis/kruskal_count_fixed_vis'] = pd.DataFrame(list(count_fixed_vis), columns=['pval', 'ci'],
C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\1713287464.py:30: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests'], dtype='object')]

  store[f'{vis_class}vis/kruskal_count_fixed_ori'] = pd.DataFrame(list(count_fixed_ori), columns=['pval', 'ci'],
C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\1713287464.py:32: PerformanceWarning: 
your performance may 

Fixed vs Controls Visual 95% CI: (-1.435032834326182e-07, 2.1054361932320198e-06)
Fixed vs Controls Visual Raw: (0.000268196058097534, array([0.00634131, 0.00634131, 0.78859931]))
Fixed vs Controls Orientation 95% CI: (-6.521874949468022e-23, 4.480354898732437e-22)
Fixed vs Controls Orientation Raw: (5.443136292223412e-08, array([5.21956459e-05, 2.30637753e-05, 9.07642115e-01]))
Fixed vs Controls Direction 95% CI: (-2.8550909326418946e-13, 4.530296065432015e-11)
Fixed vs Controls Direction Raw: (1.6802916286478037e-05, array([0.00106744, 0.00106744, 0.73690399]))


C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\1713287464.py:28: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests'], dtype='object')]

  store[f'{vis_class}vis/kruskal_count_fixed_vis'] = pd.DataFrame(list(count_fixed_vis), columns=['pval', 'ci'],
C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\1713287464.py:30: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests'], dtype='object')]

  store[f'{vis_class}vis/kruskal_count_fixed_ori'] = pd.DataFrame(list(count_fixed_ori), columns=['pval', 'ci'],
C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\1713287464.py:32: PerformanceWarning: 
your performance may 

Fixed vs Controls Visual 95% CI: (-5.160011221974568e-07, 3.2973917949911603e-06)
Fixed vs Controls Visual Raw: (0.000268196058097534, array([0.00634131, 0.00634131, 0.78859931]))
Fixed vs Controls Orientation 95% CI: (-3.4360128459774927e-22, 2.3666532173430634e-21)
Fixed vs Controls Orientation Raw: (5.443136292223412e-08, array([5.21956459e-05, 2.30637753e-05, 9.07642115e-01]))
Fixed vs Controls Direction 95% CI: (-4.613149888604364e-10, 2.1794247071877005e-09)
Fixed vs Controls Direction Raw: (1.6802916286478037e-05, array([0.00106744, 0.00106744, 0.73690399]))


C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\1713287464.py:28: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests'], dtype='object')]

  store[f'{vis_class}vis/kruskal_count_fixed_vis'] = pd.DataFrame(list(count_fixed_vis), columns=['pval', 'ci'],
C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\1713287464.py:30: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests'], dtype='object')]

  store[f'{vis_class}vis/kruskal_count_fixed_ori'] = pd.DataFrame(list(count_fixed_ori), columns=['pval', 'ci'],
C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\1713287464.py:32: PerformanceWarning: 
your performance may 

### Free Kruskal Wallis

In [30]:
# Do a Kruskal-Wallis test with Dunn posthoc
for vis_class in vis_classes:

    # Visual
    count_free_vis = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                    'count_vis_resp_free', 'visual',   
                                    kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
    print('Free vs Controls Visual 95% CI:', count_free_vis[1][1])
    print('Free vs Controls Visual Raw:', count_free_vis[0])

    # Orientation
    count_free_ori = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                    'count_vis_resp_free', 'orientation',
                                    kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
    print('Free vs Controls Orientation 95% CI:', count_free_ori[1][1])
    print('Free vs Controls Orientation Raw:', count_free_ori[0])

    # Direction
    count_free_dir = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                    'count_vis_resp_free', 'direction',
                                    kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
    print('Free vs Controls Direction 95% CI:', count_free_dir[1][1])
    print('Free vs Controls Direction Raw:', count_free_dir[0])

    # Save the data
    with pd.HDFStore(output_path, 'a') as store:
        store[f'{vis_class}vis/kruskal_count_free_vis'] = pd.DataFrame(list(count_free_vis), columns=['pval', 'ci'],
                                                    index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T
        store[f'{vis_class}vis/kruskal_count_free_ori'] = pd.DataFrame(list(count_free_ori), columns=['pval', 'ci'],
                                                    index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T
        store[f'{vis_class}vis/kruskal_count_free_dir'] = pd.DataFrame(list(count_free_dir), columns=['pval', 'ci'],
                                                    index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T

Free vs Controls Visual 95% CI: (-3.717080224392362e-07, 3.199992084061008e-06)
Free vs Controls Visual Raw: (0.0003101948186057097, array([0.01768822, 0.00235345, 0.73407518]))
Free vs Controls Orientation 95% CI: (2.1672563806664647e-07, 2.1336181263164084e-06)
Free vs Controls Orientation Raw: (0.001265652736278355, array([0.06810468, 0.00383673, 0.5248231 ]))
Free vs Controls Direction 95% CI: (-5.988030517330385e-09, 1.1148772314494695e-07)
Free vs Controls Direction Raw: (0.0002634136057045376, array([0.08240132, 0.00055088, 0.28396112]))


C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\2513833730.py:27: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests'], dtype='object')]

  store[f'{vis_class}vis/kruskal_count_free_vis'] = pd.DataFrame(list(count_free_vis), columns=['pval', 'ci'],
C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\2513833730.py:29: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests'], dtype='object')]

  store[f'{vis_class}vis/kruskal_count_free_ori'] = pd.DataFrame(list(count_free_ori), columns=['pval', 'ci'],
C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\2513833730.py:31: PerformanceWarning: 
your performance may suff

Free vs Controls Visual 95% CI: (-1.5544418439959242e-05, 6.532984985334846e-05)
Free vs Controls Visual Raw: (0.0003101948186057097, array([0.01768822, 0.00235345, 0.73407518]))
Free vs Controls Orientation 95% CI: (1.5847544321524241e-07, 1.7040130783623212e-06)
Free vs Controls Orientation Raw: (0.001265652736278355, array([0.06810468, 0.00383673, 0.5248231 ]))
Free vs Controls Direction 95% CI: (-5.855556674817974e-09, 8.941747961867321e-08)
Free vs Controls Direction Raw: (0.0002634136057045376, array([0.08240132, 0.00055088, 0.28396112]))


C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\2513833730.py:27: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests'], dtype='object')]

  store[f'{vis_class}vis/kruskal_count_free_vis'] = pd.DataFrame(list(count_free_vis), columns=['pval', 'ci'],
C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\2513833730.py:29: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests'], dtype='object')]

  store[f'{vis_class}vis/kruskal_count_free_ori'] = pd.DataFrame(list(count_free_ori), columns=['pval', 'ci'],
C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\2513833730.py:31: PerformanceWarning: 
your performance may suff

Free vs Controls Visual 95% CI: (-3.072774394837871e-07, 5.655235082075863e-06)
Free vs Controls Visual Raw: (0.0003101948186057097, array([0.01768822, 0.00235345, 0.73407518]))
Free vs Controls Orientation 95% CI: (1.0210663926774006e-07, 7.709921977413506e-07)
Free vs Controls Orientation Raw: (0.001265652736278355, array([0.06810468, 0.00383673, 0.5248231 ]))
Free vs Controls Direction 95% CI: (-1.374265081667947e-07, 5.819888873641996e-07)
Free vs Controls Direction Raw: (0.0002634136057045376, array([0.08240132, 0.00055088, 0.28396112]))


C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\2513833730.py:27: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests'], dtype='object')]

  store[f'{vis_class}vis/kruskal_count_free_vis'] = pd.DataFrame(list(count_free_vis), columns=['pval', 'ci'],
C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\2513833730.py:29: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests'], dtype='object')]

  store[f'{vis_class}vis/kruskal_count_free_ori'] = pd.DataFrame(list(count_free_ori), columns=['pval', 'ci'],
C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\2513833730.py:31: PerformanceWarning: 
your performance may suff

# Running Modulated

## Fixed

In [31]:
# Do a Kruskal-Wallis test with Dunn posthoc

# Running Modulated
frac_fixed_run_mod = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                      'frac_run_mod_fixed', 'run_modulated',
                                      kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
print('Fixed vs Controls Run Mod. 95% CI', frac_fixed_run_mod[1][1])
print('Fixed vs Controls Run Mod. 95% CI - Corrected:', frac_fixed_run_mod[-1][1])
print('Fixed vs Controls Run Mod. Raw:', frac_fixed_run_mod[0])

# Running and Visually Modulated
frac_fixed_vis_run_mod = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                        'frac_run_mod_fixed', 'vis_run_modulated',
                                        kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
print('Fixed vs Controls Vis. + Run Mod. 95% CI', frac_fixed_vis_run_mod[1][1])
print('Fixed vs Controls Vis. + Run Mod. 95% CI - Corrected:', frac_fixed_vis_run_mod[-1][1])
print('Fixed vs Controls Vis. + Run Mod. Raw:', frac_fixed_vis_run_mod[0])

# Save the data
with pd.HDFStore(output_path, 'a') as store:
    store[f'kruskal_frac_fixed_run_mod'] = pd.DataFrame(list(frac_fixed_run_mod), columns=['pval', 'ci'],
                                                  index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T
    store[f'kruskal_frac_fixed_vis_run_mod'] = pd.DataFrame(list(frac_fixed_vis_run_mod), columns=['pval', 'ci'],
                                                  index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T

Fixed vs Controls Run Mod. 95% CI (0.363631191322667, 0.39848591448664433)
Fixed vs Controls Run Mod. 95% CI - Corrected: [[0.64458453 0.68452718]
 [0.66394885 0.70255602]
 [0.57684678 0.62153293]]
Fixed vs Controls Run Mod. Raw: (0.9442514760228817, array([1., 1., 1.]))
Fixed vs Controls Vis. + Run Mod. 95% CI (0.47192512216909527, 0.5088561331687305)
Fixed vs Controls Vis. + Run Mod. 95% CI - Corrected: [[0.72069727 0.76023083]
 [0.73474963 0.7728401 ]
 [0.7280345  0.76625019]]
Fixed vs Controls Vis. + Run Mod. Raw: (0.9930991992130431, array([1., 1., 1.]))


C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\193980662.py:21: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests'], dtype='object')]

  store[f'kruskal_frac_fixed_run_mod'] = pd.DataFrame(list(frac_fixed_run_mod), columns=['pval', 'ci'],
C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\193980662.py:23: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests'], dtype='object')]

  store[f'kruskal_frac_fixed_vis_run_mod'] = pd.DataFrame(list(frac_fixed_vis_run_mod), columns=['pval', 'ci'],


In [32]:
# with pd.HDFStore(output_path, 'a') as store:
#     a = store['kruskal_frac_free_run_mod']
#     print(a.loc['ci', 'corrected_bootstrap_tests'])

## Free

In [33]:
# Do a Kruskal-Wallis test with Dunn posthoc

# Running Modulated
frac_free_run_mod = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                      'frac_run_mod_free', 'run_modulated',
                                      kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
print('Free vs Controls Run Mod. 95% CI', frac_free_run_mod[1][1])
print('Free vs Controls Run Mod. 95% CI - Corrected:', frac_free_run_mod[-1][1])
print('Free vs Controls Run Mod. Raw:', frac_free_run_mod[0])

# Running and Visually Modulated
frac_free_vis_run_mod = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                        'frac_run_mod_free', 'vis_run_modulated',
                                        kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
print('Free vs Controls Vis. + Run Mod. 95% CI', frac_free_vis_run_mod[1][1])
print('Free vs Controls Vis. + Run Mod. 95% CI - Corrected:', frac_free_vis_run_mod[-1][1])
print('Free vs Controls Vis. + Run Mod. Raw:', frac_free_vis_run_mod[0])

# Save the data
with pd.HDFStore(output_path, 'a') as store:
    store[f'kruskal_frac_free_run_mod'] = pd.DataFrame(list(frac_free_run_mod), columns=['pval', 'ci'],
                                                  index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T
    store[f'kruskal_frac_free_vis_run_mod'] = pd.DataFrame(list(frac_free_vis_run_mod), columns=['pval', 'ci'],
                                                  index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T

Free vs Controls Run Mod. 95% CI (0.015649883927561285, 0.02216972132084549)
Free vs Controls Run Mod. 95% CI - Corrected: [[0.39599727 0.43310747]
 [0.02647913 0.03782461]
 [0.08540729 0.10532919]]
Free vs Controls Run Mod. Raw: (0.20904135793189865, array([0.70912232, 0.23373511, 0.66223755]))
Free vs Controls Vis. + Run Mod. 95% CI (0.2648771404734779, 0.29891421431011866)
Free vs Controls Vis. + Run Mod. 95% CI - Corrected: [[0.55206425 0.59465317]
 [0.41093752 0.45817261]
 [0.57458274 0.61544986]]
Free vs Controls Vis. + Run Mod. Raw: (0.6308430619522412, array([1., 1., 1.]))


C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\602213168.py:21: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests'], dtype='object')]

  store[f'kruskal_frac_free_run_mod'] = pd.DataFrame(list(frac_free_run_mod), columns=['pval', 'ci'],
C:\Users\mmccann\AppData\Local\Temp\ipykernel_29592\602213168.py:23: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests'], dtype='object')]

  store[f'kruskal_frac_free_vis_run_mod'] = pd.DataFrame(list(frac_free_vis_run_mod), columns=['pval', 'ci'],
